In [4]:
import requests
import pandas as pd
import time
import os

preprocessed_df = pd.read_csv("../DBtrainrides_baseline_data.csv")

incomplete_df = preprocessed_df[preprocessed_df[["lat","long"]].isna().all(axis=1)]
df_sorted = incomplete_df.sort_values(by=['ID_Base', 'stop_number'])

# Shift the 'last_station' within each 'ID_Base' to get the next stop's 'last_station' in the current row
df_sorted['clear_station_name'] = df_sorted.groupby('ID_Base')['last_station'].shift(-1)
df_sorted

C:\Users\rapha\AppData\Local\Temp\ipykernel_18636\271306790.py:6: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  preprocessed_df = pd.read_csv("../DBtrainrides_baseline_data.csv")


,Unnamed: 0,ID_Base,ID_Timestamp,stop_number,line,starting_station_IBNR,zip,last_station,IBNR,long,...,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,canceled,info_present,transformed_info_message,clear_station_name
2043731,2043731,-9222587613942629277,2407081000,1,RB4,8010126,7545,greiz,8010140.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,greiz
2043732,2043732,-9222587613942629277,2407091000,1,RB4,8010126,7545,greiz,8010140.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,greiz
2043733,2043733,-9222587613942629277,2407101000,1,RB4,8010126,7545,greiz,8010140.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,greiz
2043734,2043734,-9222587613942629277,2407111000,1,RB4,8010126,7545,greiz,8010140.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,greiz
2043735,2043735,-9222587613942629277,2407121000,1,RB4,8010126,7545,greiz,8010140.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,greiz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440402,440402,9221732241641229965,2407101114,12,RB20,8000596,77767,zusenhofen,8006678.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,zusenhofen
440414,440414,9221732241641229965,2407111114,12,RB20,8000596,77767,zusenhofen,8006678.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,zusenhofen
440426,440426,9221732241641229965,2407121114,12,RB20,8000596,77767,zusenhofen,8006678.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,zusenhofen
440438,440438,9221732241641229965,2407131114,12,RB20,8000596,77767,zusenhofen,8006678.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,No message,zusenhofen


In [ ]:
# Define the function to get coordinates using Here API
def get_coordinates_here(station_name, api_key):
    url = f"https://geocode.search.hereapi.com/v1/geocode?q={station_name}&apiKey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['items']:
            location = data['items'][0]['position']
            return location['lat'], location['lng']
    return None, None

# API Key for Here API
api_key = os.environ.get("HERE_API_KEY")

# Initialize an empty list to store the results
results = []

# Iterate over unique station names in the 'clear_station_name' column
for idx, station_name in enumerate(df_sorted["clear_station_name"].unique()):
    # Get latitude and longitude for each station
    lat, long = get_coordinates_here(station_name, api_key)

    # Append the results to the list
    results.append({'clear_station_name': station_name, 'lat': lat, 'long': long})

    # Backup every 50 API requests
    if (idx + 1) % 50 == 0:
        # Create DataFrame and save to disk as a CSV
        temp_df = pd.DataFrame(results)
        temp_df.to_csv(f"station_coordinates_backup_{idx + 1}.csv", index=False)
        print(f"Backup saved after {idx + 1} requests.")

    # Delay to avoid hitting the API too frequently
    time.sleep(1)  # Adjust the sleep time based on API rate limits

# Final DataFrame containing all station names and coordinates
final_df = pd.DataFrame(results)

# Save the final DataFrame to disk
final_df.to_csv("station_coordinates_final.csv", index=False)
print("Final backup saved.")

In [ ]:
df = pd.read_csv("station_coordinates_final.csv")

# Iterate over the rows where lat and long are NaN
for idx, row in df[df["lat"].isna() & df["long"].isna()].iterrows():
    station_name = row["clear_station_name"]
    print(f"Station: {station_name}")

    # Get user input for latitude and longitude
    try:
        lat = float(input(f"Enter latitude for '{station_name}': "))
        long = float(input(f"Enter longitude for '{station_name}': "))

        # Update the DataFrame with user-provided values
        df.at[idx, "lat"] = lat
        df.at[idx, "long"] = long
        print(f"Updated {station_name} with lat: {lat} and long: {long}\n")

    except ValueError:
        print("Invalid input. Please enter numeric values for latitude and longitude.\n")
        continue

# Display the updated DataFrame
print("\nUpdated DataFrame:")
print(df)

# Optionally, save the DataFrame to disk
df.to_csv("station_coordinates_final_manually_updated.csv", index=False)
print("Updated DataFrame saved to 'station_coordinates_final_manually_updated.csv'")